# Motivation

We need to have a very good method of selecting resonator lengths such that we minimize the time it takes to fabricate suitable frequency-targeted qubit devices for readout with an amplifier. We must also ensure that these devices are spaced in frequency enough such that individual readout resonators are easily resolved in frequency and phase space for a given coupling rate $\kappa$. 

# Method
## Capacitance and Inductance of Bare CPW (no crossovers)

Using the geometry alone, one is able to calculate the phase velocity of the CPW by finding the the capacitance per unit length. (See [Goppel](http://arxiv.org/pdf/0807.4094v1.pdf) and [Gupta](https://www.scribd.com/doc/112426565/Gupta-Et-Al-1996-Microstrip-Lines-and-Slotlines-2nd-Ed) (p. 382) for details.) The calculation to be done are

The total line capacitance per unit length (Gupta 7.8c) becomes

\begin{align}
C_l &= 2\epsilon_0\left(\epsilon_r+1\right)\frac{K(k_0)}{K(k'_0)}
\end{align}

where $K(m)$ is the complete elliptic integral of the first kind and $k_0$ is a geometry dependent terms

\begin{align}
k_0 &= \frac{w}{w+2s},\\
k'_0 &= \sqrt{1-k_0^2}.
\end{align}

Using only the geometry and materials information, we are able to calculate the characteristic impedance of the CPW along with the phase velocity and relative permittivity of the CPW.

## Lumped Element Approximation (LEA)

We know that the resonance frequency of the cavity will be pulled by adding in extra capacitances (grounding straps, the coupling capacitance $C_{\kappa}$, the qubit pocket, etc.) and inductances (say from coupling in inductively). The frequency that we specify in the function `calcQuarterWavelength()` below is the frequency we desire *after* taking these capacitances into account.

For now just the basics: use the desired frequency $\omega'$ and calculated $Z_0$ of the CPW resonator to calculate an effective $L_{LEA}$,$C_{LEA}$ (Pozar 6.34),  resonant circuit. This lumped element circuit will ring at the target frequency $\omega'$ following

\begin{align}
\omega' &= \frac{1}{\sqrt{L_{tot}C_{tot}}} \mbox{ where}\\
C_{tot} &= C_{LEA} + C_{parasitic} \mbox{ and}\\
L_{tot} &= L_{LEA} + L_{parasitic}.
\end{align}

The desired action now is to strip the parasitics out and calculate $\omega_0$, the original frequency of the resonator before it gets pulled down. From Pozar,


\begin{align}
C_{LEA} &= \frac{\pi}{4\omega_0Z_0},\\
L_{LEA} &= \frac{1}{\omega_0^2C_{LEA}},\\
\end{align}

which can be used to find

\begin{align}
\omega_0 &= \left(\frac{1}{2}\left(-\frac{1}{\delta\omega} + \sqrt{-4C_{parasitic}L_{parasitic}+\frac{1}{(\delta\omega)^2} + \frac{4}{\omega'^2}}\right )\right)^{-1}
\end{align}

where

\begin{align}
\frac{1}{\delta\omega}&=\frac{L_{parasitic}\pi}{4Z_0}+\frac{4C_{parasitic}Z_0}{\pi}.
\end{align}

(n.b.: in the $\pm$ of solving the quadratic, the $-$ is non-physical, so the function below only uses $+$.) After we have the non-pulled frequency of the resonator, we can find the half-wavelength by using the already calculated phase velocity set by the geometry as

\begin{align}
\frac{\lambda}{4} &= \frac{\pi v_{ph}}{2\omega_0}.
\end{align}


# Code that isn't very pythonic

In [2]:
# Import extra modules as needed
import numpy as np
import scipy as sp
import peakutils
import re
import os
from collections import namedtuple

from scipy.constants import speed_of_light,mu_0,epsilon_0,pi

#########################################################################
### Function that returns the quarter-wavelength given set of parameters ###
#########################################################################
def calcQuarterWavelength(CPW_w, CPW_s, LEA_f0, SUB_er, Cp, Lp):
    # This function calculates the length of a resonator given a CPW geometry
    # and amount of parasitic capacitance (any capacitance to ground outside
    # of the self-capacitance of the CPW itself) and inductance.
    
    # CPW_w = width of the center trace
    # CPW_s = width of gap from center trace to ground plane
    # LEA_f0 = target frequency of resonator after accounting for parasitic capacitances
    # SUB_er = relative permittivity of substrate
    # STR_er = relative permittivity of oxide used in grounding straps over the CPW
    # Cp = extra capacitances beside those from purely the geometry of the CPW
    # Lp = extra inductances beside those from purely the geometry of the CPW
    
    ####################################
    ### Calculate Z0 of the geometry ###
    ####################################
    
    # Elliptic integral nonsense
    CPW_k0 = CPW_w / float(CPW_w + 2*CPW_s)
    CPW_kp = np.sqrt(1-CPW_k0**2)

    # Note that the argument is squared. This is due to the difference in how Gupta
    # defines K(k) and how the function is defined in SciPy.
    CPW_K0 = sp.special.ellipk(CPW_k0**2)
    CPW_Kp = sp.special.ellipk(CPW_kp**2)

    # Capacitance density of CPW to the substrate only.
    # Note: this isn't required to perform the calculation at hand, but is useful for troubleshooting.
    CPW_Csub = 2*epsilon_0*(CPW_K0 / CPW_Kp)

    # Total capacitance per unit length of the CPW geometry (substrate and air included)
    CPW_Cl = 2*epsilon_0*(SUB_er + 1) * (CPW_K0 / CPW_Kp)

    # Total capacitance per unit length replacing substrate with air
    CPW_Cl_air = 4*epsilon_0 * (CPW_K0 / CPW_Kp)

    # Effective relative permittivity of CPW geometry
    CPW_er = CPW_Cl / CPW_Cl_air

    # Phase velocity of this geometry
    CPW_vph = speed_of_light / np.sqrt(CPW_er)

    # CPW Impedance
    CPW_z0 = 1 / (CPW_vph * CPW_Cl)

    ################################################
    ### Calculate f0 of resonator w/o Parasitics ###
    ################################################
    
    LEA_om = 2 * pi * LEA_f0
    
    # From the lumped element model outlined above
    #RES_om = CPW_z0 * Cparasitic * LEA_om**2 / pi + LEA_om*np.sqrt((CPW_z0*Cparasitic*LEA_om/pi)**2+1)
    invDelOm = Lp * pi / float(4 * CPW_z0) + 4 * Cp * CPW_z0 / (pi)
    RES_om = (1/float(2)*(-invDelOm + np.sqrt(-4*Cp*Lp + invDelOm**2 + 4/(LEA_om**2))))**(-1)
    
    # Calculate CPW equivalent lumped elements
    LEA_C = pi / (4 * CPW_z0 * RES_om)
    LEA_L = 1 / (RES_om**2 * LEA_C)
    
    # Go from angular frequency to wavelength
    RES_lambda = 2 * pi * CPW_vph / RES_om

    # Output result in meters as well as other parameters
    output = namedtuple('CPWOutput','quarterlength z0 vph f0 fp Cp Lp er C L')
    CPW = output(RES_lambda/float(4), CPW_z0, CPW_vph, RES_om/(2*pi), LEA_f0,Cp,Lp, CPW_er, LEA_C, LEA_L)
    return CPW

# Coupling Rates

We know that the coupling to the shorted end of the resonator will result in an additional inductance to the circuit above. The coupling quality will be 

\begin{align}
Q_c &= \frac{2 Z_0 L_{LEA}}{\omega_0 M^2},
\end{align}

where $M$ is the mutual inductance between the resonator and the common feedline.

## More non-pythonic code

In [3]:
def calcMutual(Z0, L, om, Qc):
    # Function to calculate the mutual inductance given resonator impedance, total inductance, frequency, and coupling.
    totalMutual = np.sqrt(2 * Z0 * L / (om * Qc))
    return totalMutual

In [4]:
### Resonator Parameters ###

# Center trace width (um)
CPW_w = 10

# Gap from center trace to GND (symmetric on both sides) (um)
CPW_s = 6

# Target resonator frequency (Hz)
LEA_f0 = 6.6 * 10**9 

# Target coupling quality factor
RES_Qc = 1e5

# Substrate relative permittivity
SUB_er = 11.7 # intrinsic silicon

### Parasitics ###

# Qubit pocket to ground is 10 fF in series with 60 fF
LEA_Cqb = 10.6*62/(10.6+62) * 1e-15 # (F)

# Coupling capacitance to SFQ driver at the end of the resonator
LEA_Csfq = 12e-15 # (F)

# Total parasitic capacitance
Cparasitic = LEA_Cqb + LEA_Csfq

In [8]:
RES_Qc = [50000, 10000]
LEA_f0 = [6.55e9, 6.65e9]
LEA_Csfq = [0, 4e-15]
for n in range(len(RES_Qc)):
    Cparasitic = LEA_Cqb + LEA_Csfq[n]
    RES_params = calcQuarterWavelength(CPW_w,CPW_s,LEA_f0[n],SUB_er,Cparasitic,0)
    LEA_mutual = calcMutual(RES_params.z0, RES_params.L, 2*pi*RES_params.f0, RES_Qc[n])
    resonator = calcQuarterWavelength(CPW_w,CPW_s,LEA_f0[n],SUB_er,Cparasitic,LEA_mutual)
    
    print '---- Resonator {0} ----\n'.format(n)
    print 'Coupling Q:      {0}'.format(RES_Qc[n])
    print 'Target Center F: {0} GHz'.format(LEA_f0[n]/float(1e9))
    print 'Coupling Mutual: {:.2f} pH'.format(LEA_mutual*10**12)
    print 'lambda/4 length: {:.2f} um\n\n'.format(resonator.quarterlength*10**6)

---- Resonator 0 ----

Coupling Q:      50000
Target Center F: 6.55 GHz
Coupling Mutual: 8.64 pH
lambda/4 length: 4474.14 um


---- Resonator 1 ----

Coupling Q:      10000
Target Center F: 6.65 GHz
Coupling Mutual: 18.91 pH
lambda/4 length: 4367.01 um


